In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
# Paths to input and output files
csv_path = "../data/employee_data.csv"
db_path = "../data/student_roles.db"


In [4]:
# Check if files exist
assert os.path.exists(csv_path), "employee_data.csv not found"
assert os.path.exists(db_path), "student_roles.db not found"

In [5]:
# EXTRACT: Load employee data from CSV
employee_df = pd.read_csv(csv_path)

In [9]:
# TRANSFORM: Add 'status' column based on start_date
def compute_status(start_date):
    year = int(start_date.split("-")[0])
    return "active" if year <= 2023 else "pending"
employee_df["status"] = employee_df["start_date"].apply(compute_status)

In [10]:
# LOAD: Write transformed data to SQLite as 'employee_records'
conn = sqlite3.connect(db_path)
employee_df.to_sql("employee_records", conn, if_exists="replace", index=False)
conn.close()


In [12]:
# Result
print(f"ETL completed: {len(employee_df)} records loaded into 'employee_records' with status.")

ETL completed: 100 records loaded into 'employee_records' with status.


In [13]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect("../data/student_roles.db")

In [14]:
# View the table structure
pd.read_sql_query("PRAGMA table_info(employee_records);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,0
1,1,name,TEXT,0,None,0
2,2,email,TEXT,0,None,0
3,3,role,TEXT,0,None,0
4,4,department,TEXT,0,None,0
5,5,start_date,TEXT,0,None,0
6,6,status,TEXT,0,None,0


In [17]:
# Count of Active vs Pending Employees
pd.read_sql_query("""
SELECT status, COUNT(*) as count
FROM employee_records
GROUP BY status;
""", conn)


,status,count
0,active,86
1,pending,14


In [18]:
# Employees by department 
pd.read_sql_query("""
SELECT department, COUNT(*) as num_employees
FROM employee_records
GROUP BY department
ORDER BY num_employees DESC;
""", conn)


,department,num_employees
0,Product,16
1,Support,9
2,Strategy,9
3,Marketing,9
4,Logistics,9
5,IT,9
6,Engineering,8
7,Security,7
8,Finance,7
9,Sales,5


In [19]:
# Most Common Job Roles
pd.read_sql_query("""
SELECT role, COUNT(*) as frequency
FROM employee_records
GROUP BY role
ORDER BY frequency DESC;
""", conn)


,role,frequency
0,Junior Analyst,46
1,Analyst,30
2,Senior Analyst,24


In [ ]:
# Earliest and Latest Start Dates
pd.read_sql_query("""
SELECT MIN(start_date) as earliest, MAX(start_date) as latest
FROM employee_records;
""", conn)


,earliest,latest
0,2019-08-01,2024-08-01


In [21]:
# Department Breakdown by Status
pd.read_sql_query("""
SELECT department, status, COUNT(*) as count
FROM employee_records
GROUP BY department, status
ORDER BY department, status;
""", conn)


,department,status,count
0,Engineering,active,7
1,Engineering,pending,1
2,Finance,active,6
3,Finance,pending,1
4,HR,active,3
5,IT,active,8
6,IT,pending,1
7,Legal,active,3
8,Logistics,active,8
9,Logistics,pending,1


In [22]:
# Top 5 Longest-Serving Employees
pd.read_sql_query("""
SELECT id, name, start_date
FROM employee_records
ORDER BY start_date ASC
LIMIT 5;
""", conn)


,id,name,start_date
0,9,Quinn Martinez,2019-08-01
1,15,Dan Lewis,2019-08-01
2,17,Vikram Lee,2019-08-01
3,19,Xavier White,2019-08-01
4,22,Wendy White,2019-08-01


In [23]:
conn.close()